# Python 泛型与类型注解教程

欢迎来到 Python 泛型与类型注解的详细教程！本教程将引导你了解 Python 中类型提示的基础知识，并深入探讨泛型、抽象基类（ABC）以及协议（Protocol）在类型系统中的应用。

**为什么需要类型注解？**

1.  **可读性与可维护性**：明确函数参数和返回值的期望类型，使代码更易于理解。
2.  **早期错误检测**：配合静态类型检查工具（如 MyPy, Pyright/Pylance），可以在运行前发现类型不匹配的错误。
3.  **更好的IDE支持**：IDE 可以根据类型注解提供更精确的代码补全、重构和错误提示。
4.  **设计工具**：类型注解帮助思考和设计清晰的API接口。

**注意**：Python 本质上是动态类型语言，类型注解本身在运行时**不会**强制执行类型检查（除非使用特殊库如 Pydantic），它们主要服务于静态分析和开发者。

## 1. 基础类型注解

Python 3.5+ 引入了类型提示的语法（PEP 484）。

In [ ]:
# 变量注解
age: int = 30
name: str = "Alice"
pi: float = 3.14159
is_student: bool = True

# 函数注解
def greet(person_name: str) -> str:
    return f"Hello, {person_name}!"

def add(a: int, b: int) -> int:
    return a + b

print(greet("Bob"))
print(add(5, 3))

### 1.1 集合类型的注解

对于内置的集合类型，如列表、字典、元组、集合：
- Python 3.9+：可以直接使用 `list[type]`, `dict[key_type, value_type]` 等。
- Python < 3.9：需要从 `typing` 模块导入 `List`, `Dict`, `Tuple`, `Set`。

In [ ]:
from typing import List, Dict, Tuple, Set, Optional, Any, Union # 导入以备旧版本或特殊情况使用

# Python 3.9+ 风格 (推荐)
numbers: list[int] = [1, 2, 3]
scores: dict[str, float] = {"math": 90.5, "science": 88.0}
coordinates: tuple[int, int, str] = (10, 20, "Point A")
unique_names: set[str] = {"Alice", "Bob"}

# Python < 3.9 风格 (如果需要在旧版本运行)
# from typing import List, Dict, Tuple, Set
# numbers_old: List[int] = [1, 2, 3]
# scores_old: Dict[str, float] = {"math": 90.5, "science": 88.0}
# coordinates_old: Tuple[int, int, str] = (10, 20, "Point A")
# unique_names_old: Set[str] = {"Alice", "Bob"}

def process_numbers(data: list[int]) -> int:
    return sum(data)

print(process_numbers(numbers))

### 1.2 特殊类型提示

- `Optional[X]`：表示一个值可以是 `X` 类型，也可以是 `None`。等价于 `Union[X, None]` 或 Python 3.10+ 的 `X | None`。
- `Union[X, Y]`：表示一个值可以是 `X` 类型，也可以是 `Y` 类型。Python 3.10+ 可以用 `X | Y`。
- `Any`：表示一个值可以是任何类型。应谨慎使用，因为它会削弱类型检查。
- `Callable`：用于注解可调用对象（如函数）。

In [ ]:
from typing import Callable

def find_user(user_id: int) -> Optional[str]: # 或 str | None (Python 3.10+)
    if user_id == 1:
        return "Alice"
    return None

user = find_user(1)
if user:
    print(user.upper())

user_none = find_user(2)
print(user_none)

def process_item(item: Union[int, str]) -> str: # 或 int | str (Python 3.10+)
    if isinstance(item, int):
        return f"Integer: {item}"
    else:
        return f"String: {item.upper()}"

print(process_item(100))
print(process_item("hello"))

def log_message(message: Any) -> None:
    print(f"LOG: {message}")

log_message("System started")
log_message([1,2,3])

# Callable[[Arg1Type, Arg2Type], ReturnType]
def apply_operation(x: int, y: int, operation: Callable[[int, int], int]) -> int:
    return operation(x, y)

def multiply(a: int, b: int) -> int:
    return a * b

result = apply_operation(5, 3, add)       # add 函数在前面定义过
result_multiply = apply_operation(5, 3, multiply)
print(f"Add result: {result}")
print(f"Multiply result: {result_multiply}")

### 1.3 类型别名 (Type Aliases)

当类型注解变得复杂时，可以使用类型别名来简化它们。

In [ ]:
Vector = list[float]
Point = tuple[float, float]
UserScores = dict[str, list[int]]

def scale_vector(vector: Vector, scalar: float) -> Vector:
    return [x * scalar for x in vector]

my_vector: Vector = [1.0, 2.0, 3.0]
scaled_vector = scale_vector(my_vector, 2.5)
print(scaled_vector)

def get_user_average_score(scores: UserScores, user_name: str) -> Optional[float]:
    if user_name in scores and scores[user_name]:
        user_scores = scores[user_name]
        return sum(user_scores) / len(user_scores)
    return None

all_scores: UserScores = {
    "Alice": [80, 90, 85],
    "Bob": [70, 75]
}
print(get_user_average_score(all_scores, "Alice"))

## 2. 泛型 (Generics)

泛型允许我们编写可以操作多种类型的函数或类，同时保持类型安全。
关键是 `typing.TypeVar`。

### 2.1 `TypeVar`

`TypeVar` 用于声明一个类型变量。这个变量可以代表任何类型，或者在特定约束下的某些类型。

In [ ]:
from typing import TypeVar

# 创建一个类型变量 T。它可以是任何类型。
T = TypeVar('T')

# 泛型函数：输入类型和输出类型相同
def identity(item: T) -> T:
    return item

int_val = identity(10)        # T 被推断为 int
str_val = identity("hello")   # T 被推断为 str
list_val = identity([1,2,3]) # T 被推断为 list[int]

print(f"Type of int_val: {type(int_val)}, value: {int_val}")
print(f"Type of str_val: {type(str_val)}, value: {str_val}")
print(f"Type of list_val: {type(list_val)}, value: {list_val}")

# 另一个泛型函数示例
def get_first(container: list[T]) -> Optional[T]: # 或 T | None
    if container:
        return container[0]
    return None

first_num = get_first([1, 2, 3])      # T is int, returns Optional[int]
first_str = get_first(["a", "b"])    # T is str, returns Optional[str]
first_empty = get_first([])         # T is ?, returns Optional[Any] if not specified, but often inferred

print(f"First num: {first_num}")
print(f"First str: {first_str}")
print(f"First empty: {first_empty}")

### 2.2 泛型类

要创建泛型类，需要继承自 `typing.Generic`。

In [ ]:
from typing import Generic

T_co = TypeVar('T_co', covariant=True) # 协变类型变量, 常用于只读容器
T_contra = TypeVar('T_contra', contravariant=True) # 逆变类型变量，常用于只写容器或参数
# 如果不指定协变或逆变，默认为不变 (invariant)

class Box(Generic[T]): # T 是不变的
    def __init__(self, item: T):
        self._item = item

    def get_item(self) -> T:
        return self._item

    def set_item(self, item: T) -> None:
        self._item = item

    def __repr__(self) -> str:
        return f"Box({self._item!r})"

# 使用泛型类
int_box = Box[int](123)
print(int_box)
print(int_box.get_item())

str_box = Box[str]("Python")
print(str_box)
print(str_box.get_item())

# int_box.set_item("text") # Mypy会报错: Argument 1 to "set_item" of "Box" has incompatible type "str"; expected "int"
str_box.set_item("Generics")
print(str_box)

#### 关于协变 (Covariance) 和逆变 (Contravariance)

这是一个高级主题，简要说明：

- **不变 (Invariance, 默认)**: `Box[Dog]` 不是 `Box[Animal]` 的子类型，`Box[Animal]` 也不是 `Box[Dog]` 的子类型（假设 `Dog` 是 `Animal` 的子类）。
  对于可读写的容器，不变性通常是安全的。 `list[Dog]` 不是 `list[Animal]`，因为你可以向 `list[Animal]` 添加 `Cat`，但不能向 `list[Dog]` 添加 `Cat`。

- **协变 (Covariance, `covariant=True`)**: 如果 `Dog` 是 `Animal` 的子类型，那么 `Producer[Dog]` 是 `Producer[Animal]` 的子类型。这通常用于只返回 `T` 类型（生产者）的泛型。
  例如, `Sequence[Dog]` 是 `Sequence[Animal]` 的子类型，因为 `Sequence` 是只读的。

- **逆变 (Contravariance, `contravariant=True`)**: 如果 `Dog` 是 `Animal` 的子类型，那么 `Consumer[Animal]` 是 `Consumer[Dog]` 的子类型（方向相反）。这通常用于只接受 `T` 类型作为参数（消费者）的泛型。
  例如，一个函数 `Callable[[Animal], None]` 可以接受 `Dog`，所以它可以被用在期望 `Callable[[Dog], None]` 的地方。

### 2.3 约束类型变量 (Constrained TypeVar)

可以限制 `TypeVar` 只能是某些特定类型中的一个。

In [ ]:
S = TypeVar('S', str, bytes) # S 只能是 str 或 bytes

def combine(a: S, b: S) -> S:
    return a + b

str_result = combine("hello", " world") # S is str
bytes_result = combine(b"hello", b" world") # S is bytes

print(str_result)
print(bytes_result)

# combine(1, 2) # Mypy会报错: Value of type variable "S" of "combine" cannot be "int"
# combine("hello", b"world") # Mypy会报错: Type variable "S" is ambiguous

### 2.4 边界类型变量 (Bounded TypeVar)

可以限制 `TypeVar` 必须是某个类型的子类（或该类型本身）。

In [ ]:
from numbers import Number # Number 是 int, float, complex等的抽象基类

N = TypeVar('N', bound=Number) # N 必须是 Number 或其子类

def add_numbers(a: N, b: N) -> N:
    # 类型检查器知道 a 和 b 都是 Number 的子类，所以它们支持 + 操作
    return a + b

print(add_numbers(1, 2))         # N is int
print(add_numbers(1.5, 2.5))     # N is float
print(add_numbers(1, 2.5))       # N 会被推断为 float (更通用的类型)

# add_numbers("a", "b") # Mypy会报错: Value of type variable "N" of "add_numbers" cannot be "str"

## 3. 抽象基类 (Abstract Base Classes - ABCs) 与类型提示

ABCs (来自 `abc` 模块) 用于定义类接口。它们经常与类型提示一起使用，以指定函数期望接收实现了特定接口的对象。
`collections.abc` 模块包含许多有用的 ABCs，如 `Iterable`, `Sequence`, `Mapping`等。

In [ ]:
from abc import ABC, abstractmethod
from collections.abc import Sequence # 注意是 collections.abc 不是 typing

class MediaStream(ABC):
    @abstractmethod
    def play(self) -> None:
        pass

    @abstractmethod
    def stop(self) -> None:
        pass

class AudioStream(MediaStream):
    def play(self) -> None:
        print("Playing audio...")
    
    def stop(self) -> None:
        print("Stopping audio.")

class VideoStream(MediaStream):
    def play(self) -> None:
        print("Playing video...")

    def stop(self) -> None:
        print("Stopping video.")

def control_stream(stream: MediaStream) -> None:
    print(f"Controlling stream of type: {type(stream).__name__}")
    stream.play()
    stream.stop()

audio = AudioStream()
video = VideoStream()

control_stream(audio)
control_stream(video)

# 使用 collections.abc.Sequence
def print_sequence_elements(seq: Sequence[Any]) -> None:
    for i, element in enumerate(seq):
        print(f"Element {i}: {element}")

my_list = [10, 20, 30]       # list is a Sequence
my_tuple = ('a', 'b', 'c')  # tuple is a Sequence
my_string = "hello"         # str is a Sequence

print_sequence_elements(my_list)
print_sequence_elements(my_tuple)
print_sequence_elements(my_string)

# print_sequence_elements({1, 2}) # Mypy会报错, set 不是 Sequence (它没有顺序，不能通过索引访问)

## 4. 协议 (Protocols) - 结构化子类型 (Structural Subtyping)

Python 是一种“鸭子类型”语言 ("If it walks like a duck and quacks like a duck, then it must be a duck.")。
`typing.Protocol` (PEP 544, Python 3.8+) 允许我们形式化这种鸭子类型，实现结构化子类型。

一个类不需要显式继承自一个 Protocol 就能被认为是该 Protocol 的一个实现，只要它具有 Protocol 中定义的所有方法和属性（具有兼容的签名）。

In [ ]:
from typing import Protocol, runtime_checkable

# 定义一个协议
@runtime_checkable # 允许在运行时使用 isinstance() 检查 (可选)
class SupportsQuack(Protocol):
    def quack(self) -> str:
        ...

@runtime_checkable
class SupportsFly(Protocol):
    def fly(self) -> None:
        ...

# 实现协议的类 (注意：不需要显式继承)
class Duck:
    def quack(self) -> str:
        return "Quack!"
    
    def fly(self) -> None:
        print("Duck flying")

class Goose:
    def quack(self) -> str:
        return "Honk!"
    
    def fly(self) -> None:
        print("Goose flying high")

class Person:
    def speak(self) -> str:
        return "Hello!"

# 使用协议进行类型提示
def make_it_quack(obj: SupportsQuack) -> None:
    print(obj.quack())

def make_it_fly(obj: SupportsFly) -> None:
    obj.fly()

duck = Duck()
goose = Goose()
person = Person()

print("--- Making them quack ---")
make_it_quack(duck)   # Duck 实现了 SupportsQuack
make_it_quack(goose)  # Goose 也实现了 SupportsQuack
# make_it_quack(person) # Mypy会报错: Person does not have 'quack' method

print("--- Making them fly ---")
make_it_fly(duck)
make_it_fly(goose)
# make_it_fly(person) # Mypy会报错

# 运行时检查 (因为用了 @runtime_checkable)
print(f"\nIs duck a SupportsQuack? {isinstance(duck, SupportsQuack)}") # True
print(f"Is goose a SupportsQuack? {isinstance(goose, SupportsQuack)}") # True
print(f"Is person a SupportsQuack? {isinstance(person, SupportsQuack)}") # False

print(f"Is duck a SupportsFly? {isinstance(duck, SupportsFly)}") # True

### 4.1 泛型协议

协议也可以是泛型的。

In [ ]:
T_item = TypeVar('T_item')

@runtime_checkable
class SupportsGetItem(Protocol[T_item]):
    def __getitem__(self, key: int) -> T_item:
        ...

def get_element_at_index_zero(container: SupportsGetItem[T_item]) -> T_item:
    return container[0]

my_list_of_strs: list[str] = ["apple", "banana"]
my_tuple_of_ints: tuple[int, ...] = (10, 20, 30)

first_str = get_element_at_index_zero(my_list_of_strs) # T_item is str
print(f"First string: {first_str}")

first_int = get_element_at_index_zero(my_tuple_of_ints) # T_item is int
print(f"First int: {first_int}")

# 运行时检查 (需要 Python 3.8+ 和 @runtime_checkable)
print(f"Is list[str] a SupportsGetItem? {isinstance(my_list_of_strs, SupportsGetItem)}") # True

# 注意：对于泛型协议，isinstance 可能不总是能精确推断泛型参数。
# Mypy 静态检查是更可靠的。 

## 5. 静态类型检查工具

如前所述，Python 本身不强制执行类型注解。你需要使用静态类型检查工具来利用它们。

- **MyPy**: 最流行的 Python 静态类型检查器。
  ```bash
  pip install mypy
  mypy your_script.py
  ```
- **Pyright** (由微软开发，Pylance VS Code 插件的基础): 另一个快速且功能强大的类型检查器。
- **Pytype** (由谷歌开发)。

这些工具会读取你的类型注解，并报告任何不一致之处。

## 总结

类型注解和泛型是现代 Python 开发中强大的工具：
- **提高了代码质量和可维护性**：通过明确的类型声明。
- **增强了开发体验**：通过更好的 IDE 支持和早期错误检测。
- **`TypeVar`** 允许创建灵活的泛型函数和类。
- **`Generic`** 是创建泛型类的基石。
- **`ABC`** (抽象基类) 用于定义基于继承的接口（名义子类型）。
- **`Protocol`** 用于定义基于结构的接口（结构化子类型或鸭子类型），更符合 Python 的动态特性。

熟练运用这些特性，可以帮助你编写出更健壮、更易于理解和协作的 Python 代码。